# adulib.rest

In [ ]:
#|default_exp rest

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
try:
    import requests
    from urllib.parse import urljoin
    import diskcache
    import aiohttp
    import tempfile
    from asynciolimiter import Limiter
except ImportError:
    print("Install adulib[rest] to use this API.")

In [ ]:
import adulib.rest

# Async REST functions

In [ ]:
#|export
async def async_get(endpoint, params=None, headers=None):
    """Fetch data from a given RESTful API endpoint using an HTTP GET request.

    :param endpoint: The API endpoint URL (string).
    :param params: A dictionary of query parameters (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    async with aiohttp.ClientSession() as session:
        async with session.get(endpoint, params=params, headers=headers) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {"error": f"Request failed with status {response.status}", "details": await response.text()}
  
async def async_put(endpoint, data=None, headers=None):
    """Update data at a given RESTful API endpoint using an HTTP PUT request.

    :param endpoint: The API endpoint URL (string).
    :param data: A dictionary of data to send in the body of the request (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    async with aiohttp.ClientSession() as session:
        async with session.put(endpoint, json=data, headers=headers) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {"error": f"Request failed with status {response.status}", "details": await response.text()}

async def async_post(endpoint, data=None, headers=None):
    """Send data to a given RESTful API endpoint using an HTTP POST request.

    :param endpoint: The API endpoint URL (string).
    :param data: A dictionary of data to send in the body of the request (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    async with aiohttp.ClientSession() as session:
        async with session.post(endpoint, json=data, headers=headers) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {"error": f"Request failed with status {response.status}", "details": await response.text()}
    
async def async_delete(endpoint, headers=None):
    """Delete a resource at a given RESTful API endpoint using an HTTP DELETE request.

    :param endpoint: The API endpoint URL (string).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    async with aiohttp.ClientSession() as session:
        async with session.delete(endpoint, headers=headers) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {"error": f"Request failed with status {response.status}", "details": await response.text()}

In [ ]:
await async_get("https://httpbin.org/get",
    params={
        "query": "test",
        "page": 2
    },
    headers={
        "User-Agent": "MyTestClient/1.0",
        "Authorization": "Bearer testtoken123"
    }
)

{'args': {'page': '2', 'query': 'test'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Authorization': 'Bearer testtoken123',
  'Host': 'httpbin.org',
  'User-Agent': 'MyTestClient/1.0',
  'X-Amzn-Trace-Id': 'Root=1-67b1187f-042f23b12b5fc4ef5d0d3141'},
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/get?query=test&page=2'}

In [ ]:
await async_put("https://httpbin.org/put",
    data={
        "key1": "value1",
        "key2": "value2"
    },
    headers={"Content-Type": "application/json"}
)

{'args': {},
 'data': '{"key1": "value1", "key2": "value2"}',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '36',
  'Content-Type': 'application/json',
  'Host': 'httpbin.org',
  'User-Agent': 'Python/3.11 aiohttp/3.11.12',
  'X-Amzn-Trace-Id': 'Root=1-67b1187f-1ffd89da27061f3a720d2a92'},
 'json': {'key1': 'value1', 'key2': 'value2'},
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/put'}

In [ ]:
await async_post("https://httpbin.org/post",
    data={
        "key1": "value1",
        "key2": "value2"
    },
    headers={"Content-Type": "application/json"}
)

{'args': {},
 'data': '{"key1": "value1", "key2": "value2"}',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '36',
  'Content-Type': 'application/json',
  'Host': 'httpbin.org',
  'User-Agent': 'Python/3.11 aiohttp/3.11.12',
  'X-Amzn-Trace-Id': 'Root=1-67b11880-17a7e6256fd7986d6011d21c'},
 'json': {'key1': 'value1', 'key2': 'value2'},
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/post'}

In [ ]:
await async_delete("https://httpbin.org/delete",
    headers={"Content-Type": "application/json"}
)

{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '0',
  'Content-Type': 'application/json',
  'Host': 'httpbin.org',
  'User-Agent': 'Python/3.11 aiohttp/3.11.12',
  'X-Amzn-Trace-Id': 'Root=1-67b11880-73916ab226cd324226fda548'},
 'json': None,
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/delete'}

# Sync REST functions

In [ ]:
#|export
def get(endpoint, params=None, headers=None):
    """Fetch data from a given RESTful API endpoint using an HTTP GET request.

    :param endpoint: The API endpoint URL (string).
    :param params: A dictionary of query parameters (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    response = requests.get(endpoint, params=params, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status {response.status_code}", "details": response.text}
    
def post(endpoint, data=None, headers=None):
    """Send data to a given RESTful API endpoint using an HTTP POST request.

    :param endpoint: The API endpoint URL (string).
    :param data: A dictionary of data to send in the body of the request (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status {response.status_code}", "details": response.text}

def put(endpoint, data=None, headers=None):
    """Update data at a given RESTful API endpoint using an HTTP PUT request.

    :param endpoint: The API endpoint URL (string).
    :param data: A dictionary of data to send in the body of the request (default is None).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    response = requests.put(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status {response.status_code}", "details": response.text}

def delete(endpoint, headers=None):
    """Delete a resource at a given RESTful API endpoint using an HTTP DELETE request.

    :param endpoint: The API endpoint URL (string).
    :param headers: A dictionary of HTTP headers (default is None).
    :return: The JSON response as a dictionary, or an error message.
    """
    response = requests.delete(endpoint, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status {response.status_code}", "details": response.text}


In [ ]:
get("https://httpbin.org/get",
    params={
        "query": "test",
        "page": 2
    },
    headers={
        "User-Agent": "MyTestClient/1.0",
        "Authorization": "Bearer testtoken123"
    }
)

{'args': {'page': '2', 'query': 'test'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Authorization': 'Bearer testtoken123',
  'Host': 'httpbin.org',
  'User-Agent': 'MyTestClient/1.0',
  'X-Amzn-Trace-Id': 'Root=1-67b11880-3e42fd357a51e9006ea7230b'},
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/get?query=test&page=2'}

# `AsyncAPIHandler`

In [ ]:
#|export
class AsyncAPIHandler:
    def __init__(self,
                 base_url=None,
                 default_params=None,
                 default_headers=None,
                 rate_limit=None,
                 use_cache=True,
                 cache_dir=None):
        """
        :param base_url: The base URL of the API.
        :param default_params: A dictionary of default query parameters.
        :param default_headers: A dictionary of default headers.
        :param rate_limit: The rate (calls per second) limit for the API.
        :param use_cache: Whether to use caching for the API.
        :param cache_dir: The directory to store the cache.
        """
        self.base_url = base_url
        self.default_params = default_params or {}
        self.default_headers = default_headers or {}
        self.use_cache = use_cache
        self.cache_dir = cache_dir
        
        if use_cache:
            if self.cache_dir is None: self.cache_dir = tempfile.mkdtemp()
            self._cache = diskcache.Cache(self.cache_dir)
        else: self._cache = None
        
        self.rate_limit = rate_limit
        if rate_limit:
            self._rate_limiter = Limiter(rate_limit)
        else:
            self._rate_limiter = None
        
    def __get_defaults(self, method, endpoint, params, headers):
        endpoint = urljoin(self.base_url, endpoint) if endpoint else self.base_url
        params = params or {}
        headers = headers or {}
        params = {**params, **self.default_params}
        headers = {**headers, **self.default_headers}
        cache_key = f"{method}:{endpoint}:{params}:{headers}"
        return endpoint, params, headers, cache_key
    
    async def __load_cache_or_make_call(self, func, args, cache_key):
        if self._rate_limiter: await self._rate_limiter.wait()
        if self.use_cache and cache_key in self._cache:
            return self._cache[cache_key]
        else:
            result = await func(*args)
            if self.use_cache: self._cache[cache_key] = result
        return result
    
    async def get(self, endpoint=None, params=None, headers=None, **param_kwargs):
        params = params or {}
        params = {**params, **param_kwargs}
        endpoint, params, headers, cache_key = self.__get_defaults("get", endpoint, params, headers)
        return await self.__load_cache_or_make_call(async_get, (endpoint, params, headers), cache_key)
    
    async def put(self, endpoint=None, data=None, headers=None):
        endpoint, _, headers, cache_key = self.__get_defaults("put", endpoint, None, headers)
        return await self.__load_cache_or_make_call(async_put, (endpoint, data, headers), cache_key)
    
    async def post(self, endpoint=None, data=None, headers=None):
        endpoint, _, headers, cache_key = self.__get_defaults("post", endpoint, None, headers)
        return await self.__load_cache_or_make_call(async_post, (endpoint, data, headers), cache_key)
    
    async def delete(self, endpoint=None, headers=None):
        endpoint, _, headers, cache_key = self.__get_defaults("delete", endpoint, None, headers)
        return await self.__load_cache_or_make_call(async_delete, (endpoint, headers), cache_key)

In [ ]:
api_handler = AsyncAPIHandler(
    base_url="https://httpbin.org/",
    default_params={"api_key": "your_api_key"},
    default_headers={"User-Agent": "MyTestClient/1.0"},
    rate_limit=10
)

await api_handler.get("get")

{'args': {'api_key': 'your_api_key'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'MyTestClient/1.0',
  'X-Amzn-Trace-Id': 'Root=1-67b11881-5effa8fe45710d407636feb5'},
 'origin': '185.238.222.220',
 'url': 'https://httpbin.org/get?api_key=your_api_key'}